This tutorial will teach you how to convert sql queries to sqlalchemy statements. 
This repo has a `buildings.db` with two tables. `Buildings` and `Continents`.

The data is from wikipedia. [list of tallest building](https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_the_world).

We will show a sql statement, and then a sqlalchemy expression to get the same result. The sql statement can be used with the buildings.db. The required libraries can be installed with `pip install -r requirements.txt`

Let's start by getting the name of the first building.

In [1]:
import sqlalchemy
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import create_engine, MetaData, func, desc
engine = create_engine('sqlite:///buildings.db')
Session = sessionmaker()
Session.configure(bind=engine)
session = scoped_session(Session)
meta = MetaData()
meta.reflect(bind=engine)
Buildings = meta.tables["buildings"]
session.query(Buildings.c.Building).limit(1).all()

[(u'Burj Khalifa')]

We want to get the first 3 results   
`SELECT * FROM buildings LIMIT 3`


In [2]:
session.query(Buildings).limit(3).all()

[(0, u'Burj Khalifa', u'Dubai', u'UAE', 828.0, 163, 2010),
 (1, u'Shanghai Tower', u'Shanghai', u'China', 632.0, 123, 2015),
 (2,
  u'Abraj Al-Bait Clock Tower',
  u'Mecca',
  u'Saudi Arabia',
  601.0,
  120,
  2012)]

Get only the City and Building name   

    SELECT city, building FROM buildings LIMIT 3

In [3]:
session.query(Buildings.c.City, Buildings.c.Building).limit(3).all()

[(u'Dubai', u'Burj Khalifa'),
 (u'Shanghai', u'Shanghai Tower'),
 (u'Mecca', u'Abraj Al-Bait Clock Tower')]

What are the three oldest buildings in our data  

    SELECT city, building, built FROM buildings ORDER BY built LIMIT 3

In [4]:
session.query(Buildings.c.City, Buildings.c.Building, Buildings.c.Built).order_by("Built").limit(3).all()

[(u'New York City', u'Chrysler Building', 1930),
 (u'New York City', u'Empire State Building', 1931),
 (u'Chicago', u'John Hancock Center', 1969)]

What the the three highest buildings?   

    SELECT city, building, height FROM buildings ORDER BY height desc LIMIT 3

In [5]:
session.query(Buildings.c.City, Buildings.c.Building, Buildings.c.Height).order_by(desc("Height")).limit(3).all()

[(u'Dubai', u'Burj Khalifa', 828.0),
 (u'Shanghai', u'Shanghai Tower', 632.0),
 (u'Mecca', u'Abraj Al-Bait Clock Tower', 601.0)]

What are the three tallest buildings in Dubai?   

    SELECT city, building, height FROM buildings 
    WHERE city='Dubai' ORDER BY height LIMIT 3;
    

In [6]:
session.query(Buildings).filter(Buildings.c.City=="Dubai").order_by("Built").limit(3).all()

[(71, u'Burj Al Arab', u'Dubai', u'UAE', 321.0, 60, 1999),
 (39, u'Emirates Office Tower', u'Dubai', u'UAE', 355.0, 54, 2000),
 (88, u'Jumeirah Emirates Towers Hotel', u'Dubai', u'UAE', 309.0, 56, 2000)]

What are the five tallest buildings in Dubai or Chicago?

    SELECT * FROM buildings WHERE city='Dubai' 
    or city=='Chicago' ORDER BY height desc LIMIT 5;

In [7]:
session.query(Buildings).filter((Buildings.c.City=="Dubai")|(Buildings.c.City=="Chicago")).\
    order_by("Built").limit(5).all()

[(46, u'John Hancock Center', u'Chicago', u'United States', 344.0, 100, 1969),
 (44, u'Aon Center', u'Chicago', u'United States', 346.0, 83, 1973),
 (12,
  u'Willis Tower (formerly the Sears Tower)',
  u'Chicago',
  u'United States',
  442.0,
  108,
  1974),
 (93, u'Franklin Center', u'Chicago', u'United States', 307.0, 60, 1989),
 (103, u'Two Prudential Plaza', u'Chicago', u'United States', 303.0, 64, 1990)]

Among the buildings built in 20th century, which has the most floors?

    SELECT * FROM buildings WHERE built < 2000 
    ORDER BY floors DESC LIMIT 1;

In [8]:
session.query(Buildings).filter(Buildings.c.Built<2000).order_by(desc("Floors")).limit(1).all()

[(12,
  u'Willis Tower (formerly the Sears Tower)',
  u'Chicago',
  u'United States',
  442.0,
  108,
  1974)]

Among the buildings built in 20th century, which is the tallest?

    SELECT * FROM buildings WHERE built < 2000
    ORDER BY height DESC LIMIT 1;

In [9]:
session.query(Buildings).filter(Buildings.c.Built<2000).order_by(desc("Height")).limit(1).all()


[(9, u'Petronas Tower 1', u'Kuala Lumpur', u'Malaysia', 452.0, 88, 1998)]

How many different countries and cities do we have in our data?

    SELECT COUNT(DISTINCT(country)) FROM buildings;
    SELECT COUNT(DISTINCT(city)) FROM buildings;

In [10]:
session.query(func.distinct(Buildings.c.Country)).count(), session.query(func.distinct(Buildings.c.City)).count()

(17, 45)

Which cities make an apperance in top 20 tallest building list?

    SELECT DISTINCT(city) from 
    (SELECT * FROM buildings ORDER BY height DESC LIMIT 20);
    

In [11]:
top_b = session.query(Buildings).order_by(desc("Height")).limit(20).subquery()
session.query(func.distinct(top_b.c.City)).all()

[(u'Dubai'),
 (u'Shanghai'),
 (u'Mecca'),
 (u'Shenzhen'),
 (u'New York City'),
 (u'Guangzhou'),
 (u'Taipei'),
 (u'Hong Kong'),
 (u'Kuala Lumpur'),
 (u'Nanjing'),
 (u'Chicago')]

How many appearances does each conutry make in a top 100 list?

    SELECT country, COUNT(country) AS building_count 
    FROM 
    (select * from buildings order by height limit 100) 
    GROUP BY country 
    ORDER BY building_count DESC;
    
    

In [12]:
top_b = session.query(Buildings).order_by(desc("Height")).limit(100).subquery()
session.query(top_b.c.Country, func.count("*").label("building_count")).\
    group_by(top_b.c.Country).order_by(desc("building_count")).all()

[(u'China', 44),
 (u'UAE', 24),
 (u'United States', 15),
 (u'Russia', 4),
 (u'Malaysia', 3),
 (u'Saudi Arabia', 3),
 (u'Republic of China', 2),
 (u'Australia', 1),
 (u'Kuwait', 1),
 (u'North Korea', 1),
 (u'United Kingdom', 1),
 (u'Vietnam', 1)]

What is the average floors and average height, per country.
   
    SELECT country, AVG(floors), AVG(height) from 
        (SELECT * FROM buildings ORDER BY height DESC LIMIT 100)
        GROUP BY country;

In [13]:
session.query(top_b.c.Country, func.avg(top_b.c.Floors), func.avg(top_b.c.Height)).\
    group_by(top_b.c.Country).all()

[(u'Australia', 80.0, 323.0),
 (u'China', 78.18181818181819, 370.8863636363636),
 (u'Kuwait', 80.0, 413.0),
 (u'Malaysia', 77.0, 404.6666666666667),
 (u'North Korea', 105.0, 330.0),
 (u'Republic of China', 93.0, 428.5),
 (u'Russia', 81.75, 344.0),
 (u'Saudi Arabia', 88.33333333333333, 431.3333333333333),
 (u'UAE', 80.58333333333333, 365.5416666666667),
 (u'United Kingdom', 72.0, 309.0),
 (u'United States', 80.26666666666667, 363.46666666666664),
 (u'Vietnam', 72.0, 329.0)]

What is the average floors and average height, per country, sorted by height

    SELECT country, AVG(floors) as avg_floor, AVG(height) as avg_height from 
        (SELECT * FROM buildings ORDER BY height DESC LIMIT 100)
        GROUP BY country
        ORDER by avg_height DESC;

In [14]:
session.query(top_b.c.Country, func.avg(top_b.c.Floors), func.avg(top_b.c.Height).label("avg_height")).\
    group_by(top_b.c.Country).order_by(desc("avg_height")).all()

[(u'Saudi Arabia', 88.33333333333333, 431.3333333333333),
 (u'Republic of China', 93.0, 428.5),
 (u'Kuwait', 80.0, 413.0),
 (u'Malaysia', 77.0, 404.6666666666667),
 (u'China', 78.18181818181819, 370.8863636363636),
 (u'UAE', 80.58333333333333, 365.5416666666667),
 (u'United States', 80.26666666666667, 363.46666666666664),
 (u'Russia', 81.75, 344.0),
 (u'North Korea', 105.0, 330.0),
 (u'Vietnam', 72.0, 329.0),
 (u'Australia', 80.0, 323.0),
 (u'United Kingdom', 72.0, 309.0)]

What years were the buildings completed?

    SELECT built, count(built) FROM buildings GROUP BY built;

In [15]:
session.query(Buildings.c.Built, func.count("*")).group_by("Built").all()

[(1930, 1),
 (1931, 1),
 (1969, 1),
 (1973, 1),
 (1974, 1),
 (1982, 1),
 (1983, 1),
 (1989, 2),
 (1990, 2),
 (1992, 2),
 (1996, 1),
 (1997, 3),
 (1998, 3),
 (1999, 2),
 (2000, 2),
 (2001, 1),
 (2002, 1),
 (2003, 1),
 (2004, 1),
 (2005, 2),
 (2007, 4),
 (2008, 3),
 (2009, 5),
 (2010, 11),
 (2011, 9),
 (2012, 13),
 (2013, 6),
 (2014, 8),
 (2015, 32)]

Lets do some joins

    SELECT building, continent from BUILDING LEFT JOIN continents 
    ON buildings.country=continents.country LIMIT 1;

In [16]:
Continents = meta.tables["continents"]
session.query(Buildings.c.Building, Continents.c.Continent).\
    join(Continents, Continents.c.Country==Buildings.c.Country).limit(1).all()

[(u'Burj Khalifa', u'Asia')]

What is the average floor count and height on an each continent basis?

    SELECT continent, avg(floors), avg(height) FROM buildings LEFT JOIN continents 
    ON buildings.country=continents.country 
    GROUP by continent;
    

In [17]:
session.query(Continents.c.Continent, func.avg(Buildings.c.Floors), func.avg(Buildings.c.Height)).\
    join(Buildings, Continents.c.Country==Buildings.c.Country).group_by("Continent").all()

[(u' Asia', 75.57894736842105, 360.63157894736844),
 (u' Australia', 80.0, 323.0),
 (u' Europe', 79.16666666666667, 331.1666666666667),
 (u' South America', 62.0, 300.0),
 (u'Asia', 77.36363636363636, 361.2857142857143),
 (u'North America', 78.76470588235294, 356.29411764705884)]

Thank you. If you *think in sql*, I hope this tutorial allows you to transform your queries easily to SQLAlchemy equivalents.